In [173]:
import pandas as pd
from io import BytesIO
import requests
import random

def fetch_data(url, skiprows=0):
    r = requests.get(url)
    data = r.content

    return pd.read_csv(BytesIO(data), index_col=0, skiprows=skiprows)

catalog = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=0&format=csv", 2)
data_interp_catalog = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=1340046923&format=csv", 0)

completed_sets = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=1791033131&format=csv")
completed_suppl = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=399306386&format=csv")

In [174]:
categories = catalog.keys()[7:]
repl_nan = { i : False for i in catalog.keys()[3:]}
repl_checked = { i : 'x' for i in catalog.keys()[3:]}

scrubbed_cat = catalog.fillna(value=repl_nan).replace(repl_checked, True)
scrubbed_di = data_interp_catalog.fillna(value=repl_nan).replace(repl_checked, True)
scrubbed_all = pd.concat([scrubbed_di, scrubbed_cat], ignore_index=False, sort=False).drop_duplicates()
scrubbed_all

,Set Id,Local Id,Source,Type,AR,ALG,DA,GEO,ARG,FAD,...,PRS,CC,TRI,CG,MG,WP,2WP,VC,AQ,DI
Global Id,,,,,,,,,,,,,,,,,,,,,
1,A,24.01,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,A,24.02,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,A,24.03,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,A,24.04,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5,A,24.05,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,True
6,B,24.06,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7,B,24.07,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
8,B,24.08,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,C,24.09,M,MC,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [186]:
categorical_cols = scrubbed_all.columns[4:8]
ctotals = { col:scrubbed_all[col].where(scrubbed_all[col]==True).count() for col in categorical_cols }
ctotals

{'AR': 247, 'ALG': 223, 'DA': 166, 'GEO': 136}

In [189]:
skillset_cols = scrubbed_all.columns[8:-5] 
stotals = { col:scrubbed_all[col].where(scrubbed_all[col]==True).count() for col in skillset_cols }
stotals
skillset_cols

Index(['ARG', 'FAD', 'PCT', 'NP', 'DAP', 'EAR', 'EAF', 'IAV', 'FFS', 'ROW',
       'RAT', 'CT', 'DST', 'PROB', 'PRS', 'CC', 'TRI', 'CG', 'MG'],
      dtype='object')

In [190]:
misc_cols = scrubbed_all.columns[-5:]
mtotals = { col:scrubbed_all[col].where(scrubbed_all[col]==True).count() for col in misc_cols }
mtotals

{'WP': 45, '2WP': 25, 'VC': 20, 'AQ': 42, 'DI': 52}

In [191]:
completed_full = pd.concat([completed_sets, completed_suppl], ignore_index=False, sort=False).drop_duplicates()
j = scrubbed_all.join(completed_full, how='right', lsuffix='_left', rsuffix='_right', sort=False)
completed_meta = j[j['Local Id_right'].notna()]
completed_meta

,Set Id_left,Local Id_left,Source,Type,AR,ALG,DA,GEO,ARG,FAD,...,2WP,VC,AQ,DI,Set Id_right,Local Id_right,Completed,Correct,Date,Unnamed: 4
Global Id,,,,,,,,,,,,,,,,,,,,,
1,A,24.01,M,MC,False,False,True,False,False,False,...,False,False,False,True,4.0,24.01,True,True,NaN,NaN
2,A,24.02,M,MC,False,False,True,False,False,False,...,False,False,False,True,2.0,24.02,True,True,NaN,NaN
3,A,24.03,M,MC,False,False,True,False,False,False,...,False,False,False,True,4.0,24.03,True,True,NaN,NaN
5,A,24.05,M,MC,False,False,True,False,False,False,...,False,False,False,True,4.0,24.05,True,True,NaN,NaN
42,L,24.42,M,MC,False,False,True,False,False,False,...,False,False,False,True,3.0,24.42,True,True,NaN,NaN
43,L,24.43,M,MC,False,False,True,False,False,False,...,False,False,False,True,3.0,24.43,True,True,NaN,NaN
44,L,24.44,M,MC,False,False,True,False,False,False,...,False,False,False,True,3.0,24.44,True,True,NaN,NaN
45,L,24.45,M,MC,False,False,True,False,False,False,...,False,False,False,True,3.0,24.45,True,True,NaN,NaN
58,NaN,7.09,M,NE,True,False,False,False,True,False,...,False,False,False,False,3.0,7.09,True,True,NaN,NaN


In [214]:
def key_based_accuracy(completed_meta, keys):
    wrong = {}
    accuracy = {}
    for cat in keys:
        df = completed_meta[completed_meta[cat] == True]
        correct = df[df['Correct'] == True].count()[cat]
        count = df.count()[cat]

        wrong[cat] = df[df['Correct'] == False]
        accuracy[cat] = (count, correct / count)
        
    return (wrong, accuracy)

In [215]:
cacc = key_based_accuracy(completed_meta, categorical_cols)[1]
cacc

{'AR': (46, 0.8260869565217391),
 'ALG': (54, 0.7777777777777778),
 'DA': (27, 0.7777777777777778),
 'GEO': (31, 0.7096774193548387)}

In [216]:
sacc = key_based_accuracy(completed_meta, skillset_cols)[1]
sacc

{'ARG': (3, 1.0),
 'FAD': (6, 0.8333333333333334),
 'PCT': (11, 0.8181818181818182),
 'NP': (11, 0.8181818181818182),
 'DAP': (6, 0.6666666666666666),
 'EAR': (12, 0.8333333333333334),
 'EAF': (16, 0.8125),
 'IAV': (13, 0.7692307692307693),
 'FFS': (9, 0.7777777777777778),
 'ROW': (7, 0.8571428571428571),
 'RAT': (11, 0.7272727272727273),
 'CT': (6, 0.6666666666666666),
 'DST': (4, 0.75),
 'PROB': (10, 0.6),
 'PRS': (6, 0.8333333333333334),
 'CC': (7, 0.5714285714285714),
 'TRI': (6, 0.5),
 'CG': (5, 1.0),
 'MG': (8, 0.875)}

In [217]:
macc = key_based_accuracy(completed_meta, misc_cols)[1]
macc

{'WP': (19, 0.7894736842105263),
 '2WP': (4, 1.0),
 'VC': (2, 0.5),
 'AQ': (5, 0.8),
 'DI': (11, 0.9090909090909091)}

In [227]:
def type_based_accuracy(completed_meta):
    accuracy = {}
    for typ in ['QC', 'MC', 'NE', 'MA']:
        df = completed_meta[completed_meta['Type'] == typ]
        correct = df[df['Correct'] == True].count()['Type']
        count = df.count()['Type']
        accuracy[typ] = (count, correct / count)
    return accuracy

In [228]:
tacc = type_based_accuracy(completed_meta)
tacc

{'QC': (52, 0.7692307692307693),
 'MC': (67, 0.8208955223880597),
 'NE': (28, 0.8214285714285714),
 'MA': (9, 0.4444444444444444)}

In [229]:
# Hardcoded from http://localhost:8888/notebooks/Documents/git/gre_analysis/Test%20Structure%20Analysis.ipynb.
# Recopy if data is added.

type_test_weights = {
    'MA': 0.0875, 
    'MC': 0.44000000000000006, 
    'NE': 0.095, 
    'QC': 0.3775
}

cat_test_weights = {
    'AR': (63, 0.37058823529411766),
    'ALG': (62, 0.36470588235294116),
    'DA': (43, 0.2529411764705882),
    'GEO': (35, 0.20588235294117646)
}

skillset_test_weights = {
    'ARG': (9, 0.052941176470588235),
     'FAD': (4, 0.023529411764705882),
     'PCT': (23, 0.13529411764705881),
     'NP': (13, 0.07647058823529412),
     'DAP': (7, 0.041176470588235294),
     'EAR': (10, 0.058823529411764705),
     'EAF': (27, 0.1588235294117647),
     'IAV': (13, 0.07647058823529412),
     'FFS': (8, 0.047058823529411764),
     'ROW': (7, 0.041176470588235294),
     'RAT': (11, 0.06470588235294118),
     'CT': (7, 0.041176470588235294),
     'DST': (4, 0.023529411764705882),
     'PROB': (11, 0.06470588235294118),
     'PRS': (5, 0.029411764705882353),
     'CC': (6, 0.03529411764705882),
     'TRI': (13, 0.07647058823529412),
     'CG': (9, 0.052941176470588235),
     'MG': (4, 0.023529411764705882)
}

misc_test_weights = {
    'WP': (15, 0.08823529411764706),
    '2WP': (5, 0.029411764705882353),
    'VC': (2, 0.011764705882352941),
    'AQ': (0, 0.0),
    'DI': (26, 0.15294117647058825)
}

In [230]:
# Note these values are psuedo-weighted. Questions can have multiple categories, so the weights won't add up to 1.
# The absolute values of each category are meaningless, but they are proportionate to the error rate and likelihood of 
# seeing the category on a test, thus providing some semblence of relative importance.
def weighted_accuracy(accuracy, test_weight):
    new_dict = {}
    for k in accuracy:
        new_dict[k] = (1-accuracy[k][1]) * test_weight[k][1]
    return new_dict

def weighted_sample_size(accuracy, test_weight):
    new_dict = {}
    for k in accuracy:
        new_dict[k] = test_weight[k][1] / accuracy[k][0]
    return new_dict

In [231]:
cweighted = weighted_accuracy(cacc, cat_test_weights)
sweighted = weighted_accuracy(sacc, skillset_test_weights)
mweighted = weighted_accuracy(macc, misc_test_weights)
sorted(list(cweighted.items()), key=lambda tup: tup[1], reverse=True)
sorted(list(sweighted.items()), key=lambda tup: tup[1], reverse=True)
sorted(list(mweighted.items()), key=lambda tup: tup[1], reverse=True)
sorted(list(sweighted.items()) + list(mweighted.items()), key=lambda tup: tup[1], reverse=True)

[('TRI', 0.03823529411764706),
 ('EAF', 0.02977941176470588),
 ('PROB', 0.025882352941176474),
 ('PCT', 0.024598930481283414),
 ('WP', 0.018575851393188854),
 ('IAV', 0.01764705882352941),
 ('RAT', 0.01764705882352941),
 ('CC', 0.015126050420168069),
 ('DI', 0.013903743315508027),
 ('NP', 0.013903743315508019),
 ('DAP', 0.013725490196078433),
 ('CT', 0.013725490196078433),
 ('FFS', 0.01045751633986928),
 ('EAR', 0.009803921568627449),
 ('ROW', 0.005882352941176472),
 ('DST', 0.0058823529411764705),
 ('VC', 0.0058823529411764705),
 ('PRS', 0.0049019607843137246),
 ('FAD', 0.0039215686274509795),
 ('MG', 0.0029411764705882353),
 ('ARG', 0.0),
 ('CG', 0.0),
 ('2WP', 0.0),
 ('AQ', 0.0)]

In [232]:
sorted(list(cacc.items()), key=lambda tup: tup[1][0])
sorted(list(sacc.items()), key=lambda tup: tup[1][0])
sorted(list(sacc.items()) + list(macc.items()), key=lambda tup: tup[1][0])

[('VC', (2, 0.5)),
 ('ARG', (3, 1.0)),
 ('DST', (4, 0.75)),
 ('2WP', (4, 1.0)),
 ('CG', (5, 1.0)),
 ('AQ', (5, 0.8)),
 ('FAD', (6, 0.8333333333333334)),
 ('DAP', (6, 0.6666666666666666)),
 ('CT', (6, 0.6666666666666666)),
 ('PRS', (6, 0.8333333333333334)),
 ('TRI', (6, 0.5)),
 ('ROW', (7, 0.8571428571428571)),
 ('CC', (7, 0.5714285714285714)),
 ('MG', (8, 0.875)),
 ('FFS', (9, 0.7777777777777778)),
 ('PROB', (10, 0.6)),
 ('PCT', (11, 0.8181818181818182)),
 ('NP', (11, 0.8181818181818182)),
 ('RAT', (11, 0.7272727272727273)),
 ('DI', (11, 0.9090909090909091)),
 ('EAR', (12, 0.8333333333333334)),
 ('IAV', (13, 0.7692307692307693)),
 ('EAF', (16, 0.8125)),
 ('WP', (19, 0.7894736842105263))]

In [130]:
sum(map(lambda x: x[1][0], sacc.items()))

157